# ASX Energy Asian Option Pricing Demo\n\nEnd-to-end example: load market data, calibrate Heston (if QuantLib is available), run Monte Carlo for Heston and VG/NIG, store the run, and render a dashboard.

In [ ]:
import logging\nfrom pathlib import Path\n\nimport numpy as np\n\nfrom asians_pricer import (\n    AsianOption,\n    GreekEngine,\n    HestonParams,\n    LevyMonteCarloEngine,\n    NIGParams,\n    VarianceGammaParams,\n    VectorizedHestonEngine,\n    configure_logging,\n    record_run,\n)\nfrom asians_pricer.visualization.plotting import render_dashboard\n\nconfigure_logging(logging.INFO)

## 1. Load market data (placeholder)\n\nReplace with real vanilla option quotes. Here we mock a flat vol surface and forward price.

In [ ]:
spot = 105.0\nrisk_free = 0.03\nimplied_vol = 0.6  # placeholder daily vol for calibration stub\nvanilla_quotes = [  # (strike, expiry_years, implied_vol)\n    (90, 0.25, implied_vol),\n    (100, 0.25, implied_vol * 0.95),\n    (110, 0.25, implied_vol * 1.05),\n]

## 2. Calibrate Heston parameters (if QuantLib is installed)\n\nFalls back to preset parameters if QuantLib is unavailable.

In [ ]:
try:\n    from datetime import date\n    from asians_pricer.models import HestonCalibrator\n\n    calibrator = HestonCalibrator(date.today(), spot_price=spot, risk_free_rate=risk_free)\n    heston_params = calibrator.calibrate(vanilla_quotes)\n    logging.info("Calibrated Heston params: %s", heston_params)\nexcept Exception as exc:\n    logging.warning("QuantLib calibration unavailable, using defaults: %s", exc)\n    heston_params = HestonParams(v0=0.04, kappa=2.0, theta=0.09, sigma=0.5, rho=-0.4)

## 3. Define the Asian option contract

In [ ]:
option = AsianOption(strike=100.0, maturity=0.25, is_call=True)

## 4. Price with Heston Monte Carlo and Greeks

In [ ]:
heston_engine = VectorizedHestonEngine(params=heston_params, risk_free_rate=risk_free, steps_per_year=252)\nheston_res = heston_engine.price_asian(\n    option=option,\n    S0=spot,\n    n_paths=30_000,\n    antithetic=True,\n    control_variate=True,\n    seed=42,\n    diag_samples=200,\n)\nlogging.info("Heston price: %.4f (se=%.6f)", heston_res['price'], heston_res['std_error'])\n\ngreeks = GreekEngine(heston_engine, seed=42).calculate(option=option, S0=spot, n_paths=10_000)\ngreeks

## 5. Price with Variance Gamma and NIG Monte Carlo

In [ ]:
levy_engine = LevyMonteCarloEngine(risk_free_rate=risk_free, steps_per_year=252)\n\nvg_params = VarianceGammaParams(theta=0.0, sigma=0.25, nu=0.2)\nvg_res = levy_engine.price_asian(\n    option=option,\n    S0=spot,\n    n_paths=30_000,\n    params=vg_params,\n    process="vg",\n    seed=7,\n    diag_samples=200,\n)\n\nnig_params = NIGParams(alpha=5.0, beta=-2.0, delta=0.5, mu=0.0)\nnig_res = levy_engine.price_asian(\n    option=option,\n    S0=spot,\n    n_paths=30_000,\n    params=nig_params,\n    process="nig",\n    seed=7,\n    diag_samples=200,\n)\nvg_res['price'], nig_res['price']

## 6. Store runs and render dashboards

In [ ]:
# Persist run details\nrecord_run("heston", option, heston_params, {k: v for k, v in heston_res.items() if k != "diagnostics"})\nrecord_run("variance_gamma", option, vg_params, {k: v for k, v in vg_res.items() if k != "diagnostics"})\nrecord_run("nig", option, nig_params, {k: v for k, v in nig_res.items() if k != "diagnostics"})\n\n# Render dashboards (requires plotly)\ndash_dir = Path("runs")\ndash_dir.mkdir(exist_ok=True)\nheston_dash = dash_dir / "heston_dashboard.html"\nvg_dash = dash_dir / "vg_dashboard.html"\nnig_dash = dash_dir / "nig_dashboard.html"\n\nrender_dashboard(heston_res["diagnostics"], heston_res, output_path=str(heston_dash))\nrender_dashboard(vg_res["diagnostics"], vg_res, output_path=str(vg_dash))\nrender_dashboard(nig_res["diagnostics"], nig_res, output_path=str(nig_dash))\n\nheston_dash, vg_dash, nig_dash